# BERTopic

## Install packages
Si tu run le code en local, installe d'abord torch pour CPU (avec le code suivant), sinon tu auras tous les drivers pour CUDA en installant directement BERTopic.

!pip uninstall -y tensorflow

In [ ]:
!pip install umap-learn
!pip install hdbscan
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic import BERTopic

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Embedding
On a pris cet embedding car il fonctionnait bien sur les textes avec du vocabulaire médical mais si ton cas d'utilisation n'est pas médical, n'hésite pas à en changer, tu as la liste des [modèles utilisables par `SentenceTransformer` en cliquant ici](https://huggingface.co/models?library=sentence-transformers&sort=trending&search=med).

In [ ]:
embedding_model = SentenceTransformer('abhinand/MedEmbed-large-v0.1')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

## Réduction de dimension
Si tu as peu observations, tu peux aussi réduire `n_components`, qui correspond au nombre de dimensions après la réduction de dimensions. $$n_{components} \gtrapprox \log_{10}(n_{posts})$$

In [ ]:
umap_model = UMAP(
    n_neighbors=6,
    n_components=2,  # much higher latent dimension
    min_dist=0.1,  # increased min distance
    random_state=42,
    metric='cosine'
)

## Clustering
Pour obtenir des topics plus petits, commence par diminuer le paramètre `min_cluster_size`. Si les résultats ne sont pas satisfaisants, réduis `min_samples`. Si cela ne suffit pas, diminue `n_neighbors` dans `UMAP`.

In [ ]:
hdbscan_model = HDBSCAN(
    min_cluster_size=10,  # increase min_cluster_size
    min_samples=10,  # 2X increase min_samples
    metric='euclidean',
    cluster_selection_method='eom',
    cluster_selection_epsilon=0.01,
    alpha=1.0,
    prediction_data=True
)

## Representation
`CountVectorizer` permet de découper les textes en n-uplets. Ici ce sont en mots uniques, en couples et triplets de mots. `ClassTfidfTransformer` calcule les scores TF-IDF sur les éléments donnés par `CountVectorizer`.

Cela te permettra d'avoir les mots-clefs (ou n-uplets-clefs ?) qui représentent le mieux chacun des topics calculés.

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 3))
ctfidf_model = ClassTfidfTransformer()

Si tu as un compte développeur OpenAI, tu peux utiliser l'API ChatGPT pour nommer les topics.

In [ ]:
print(openai.__version__)  # Assure-toi que c'est >=1.0.0


NameError: name 'openai' is not defined

In [ ]:


import os
os.environ["OPENAI_API_KEY"] = "sk-proj-moeirhjtmoiezrhnjtmoizehrt-32HyaGVzoEVjK9i7RPtBWtzuDQ6RYpKKT3BlbkFJTZ9DT8mlHpwQG40gvRn8CH7nKPLJ6QYkYkG2ldD7gb9k5A82ri6WohrexUf9oEcqeVw8SjYhIA" # Replace with your actual key
from openai import OpenAI

client = OpenAI()

representation_model = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)
print(representation_model.choices[0].message.content)



TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [ ]:
from bertopic.representation import OpenAI

client = openai.OpenAI(api_key="sk-proj-ytarezhtapierutpiaeurnt...")
representation_model = OpenAI(client, model="gpt-4o-mini", chat=True)

## BERTopic

In [ ]:
bert = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    # representation_model=representation_model,  # uncomment if OpenAI is used
    nr_topics='auto',
    low_memory=True,
)

## Chargement des données

In [ ]:
df = pd.read_...
docs = df['text_clean'].to_list()

## Inférence et resultats
`topics` a la même longueur que `docs` et son allignés donc tu peux ajouter un colonne à `df` contenant l'ID de topic devant chaque poste. Si un poste ne correspond à aucun topic, son ID de topic sera -1. On dit que ce poste fait partie du bruit.

In [ ]:
topics, _ = bert.fit_transform(docs)

Tu peux avoir un résumé des topics générés en obtenant un data frame.

In [ ]:
topic_list = bert.get_topic_info()